## Python script - Data preparation for Aerosol Absorbing Index

Data from CEDA Archive

ESA Multi Satellite Product

Available to access from JASMIN:
https://catalogue.ceda.ac.uk/uuid/2e656d34d016414c8d6bced18634772c

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import os
from datetime import datetime

In [2]:
dir_in = '/gws/nopw/j04/vplume/mchim/esa_aerosol_cci_data/MONTHLY/'
dir_out = '/gws/nopw/j04/vplume/mchim/esa_aerosol_cci_data/'

In [ ]:
# Read and combine monthly datafiles 
combined_dataset = xr.Dataset()
time_values = []
year_month_strings = []

for year_dir in sorted(glob.glob(os.path.join(dir_in, '*'))):
    year = os.path.basename(year_dir)
    
    nc_files = sorted(glob.glob(os.path.join(year_dir, '*.nc')))
    
    for nc_file in nc_files:
        file_name = os.path.basename(nc_file)
        year_month = file_name.split('-')[6]
        year = int(year_month[:4]) 
        month = int(year_month[4:])
        
        dataset = xr.open_dataset(nc_file)
        
        time_values.append(datetime(year, month, 1))
    
        combined_dataset = xr.concat([combined_dataset, dataset], dim='time')

combined_dataset['time'] = time_values
combined_dataset.to_netcdf(dir_out+'combined_1978_2015.nc')

In [4]:
da = xr.open_dataset(dir_out+'combined_1978_2015.nc')
da

<xarray.Dataset>
Dimensions:                           (time: 422, latitude: 180, longitude: 360)
Coordinates:
  * longitude                         (longitude) float32 -179.5 ... 179.5
  * latitude                          (latitude) float32 -89.5 -88.5 ... 89.5
  * time                              (time) datetime64[ns] 1978-11-01 ... 20...
Data variables:
    number_of_positive_observations   (time, latitude, longitude) int16 ...
    number_of_observations            (time, latitude, longitude) int16 ...
    positive_absorbing_aerosol_index  (time, latitude, longitude) float32 ...
    solar_zenith_angle                (time, latitude, longitude) float32 ...
    absorbing_aerosol_index           (time, latitude, longitude) float32 ...

In [215]:
# Global averaged aerosol absorbing index
da_annual = da.resample(time='A').mean()

weights = np.cos(np.deg2rad(da_annual.latitude))
weights.name = "weights"
temp_da = da_annual.weighted(weights)
da_annual_global = temp_da.mean(dim=['longitude','latitude'])

df = da_annual_global.to_dataframe()
df.to_csv("global_data.csv")

In [204]:
def calc_aai_region(data, lat1, lat2, lon1, lon2):

    min_lat = 90+lat1
    max_lat = 90+lat2   
    min_lon = 180+lon1   
    max_lon = 180+lon2 
    
    slice_data = data[:, min_lat:max_lat, min_lon:max_lon]
        
    weights = np.cos(np.deg2rad(slice_data.latitude))
    weights.name = "weights"
    
    temp_da = slice_data.weighted(weights)
    region_average = temp_da.mean(dim=['longitude','latitude'])

    return region_average

In [232]:
# Aerosol absorbing index averaged over Indonesia in October
indonesia_data = calc_aai_region(da.positive_absorbing_aerosol_index,-11,5,95,141)
indonesia_data = indonesia_data.sel(time=indonesia_data['time.month']== 10)

# Aerosol absorbing index averaged over Australia in December
aus_data = calc_aai_region(da.positive_absorbing_aerosol_index,-45,-10,113,154)
aus_data = aus_data.sel(time=aus_data['time.month']== 12)

df = indonesia_data.to_dataframe()
df.to_csv("indonesia_data.csv")

df = aus_data.to_dataframe()
df.to_csv("aus_data.csv")